# 지금까지 배운 내용

1. 데이터 수집
2. 데이터 전처리, 탐색, 변환

# 그럼 이제 데이터 분석을 할 차례

1. (Supervised) 후보를 설명하는 단어 추출

    - 입력변수: 뉴스에 등장한 단어의 tf-idf 값
    - 출력변수: 후보에 대한 뉴스 여부

2. (Unsupervised) 대선 주요 토픽 탐색

    - 입력데이터: 뉴스 데이터
    - 출력변수: 없음

# Load Data

In [ ]:
import json
import os

import numpy as np
import pandas as pd

In [ ]:
cand_list = ['문재인', '안철수', '유승민', '홍준표', '심상정']
columns = ['sid', 'press', 'date', 'title', 'contents', 'label', 'len_news', 'num_sent']

In [ ]:
df = pd.DataFrame(columns=columns)
seen_aid = set()

for cand in cand_list:
    dir_path = os.path.join('./data/', cand)
    
    for file_name in os.listdir(dir_path):
        sid, _, aid = file_name.replace('.json', '').split('-')
        
        if aid in seen_aid:
            try:
                df.drop(aid, inplace=True)
            except:
                pass
        else:
            seen_aid.add(aid)

            file_path = os.path.join(dir_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
        
            # add columns
            data['label'] = cand
            data['sid'] = sid
            data['date'] = data['datetime'][:10]
            data['len_news'] = len(data['contents'])
            data['num_sent'] = len(data['contents'].split('.'))
            # modify columns
            data['contents'] = data['contents'].replace('\r', '').replace('\n', '')
            
            for col in columns:
                df.loc[aid, col] = data[col]

In [ ]:
df.head(1)

In [ ]:
len(df)

# Filter some news

#### 분석에 불필요한 뉴스를 제외하기

1. 길이가 짧은 뉴스: 200자 이상만 남기기
2. 포토 뉴스
3. 속보

In [ ]:
def filter_rows(df, word_list):
    for word in word_list:
        df = df[df.apply(lambda x: word not in x.title, axis=1)]
    return df

In [ ]:
df_filtered = filter_rows(df, ['포토', '사진', '속보'])
df_filtered = df_filtered[df_filtered.len_news > 200]

In [ ]:
df_filtered.sample()

In [ ]:
df_filtered.shape

# Extarct Nouns

In [ ]:
from konlpy.tag import Twitter

twitter = Twitter()
news_noun_list = [
    twitter.nouns(news) for news in df_filtered['contents']
]

# Set Features

In [ ]:
stop_list = set('문재인 안철수 심상정 홍준표 유승민 민주당 국민의당 정의당 바른정당 자유한국당 기자 무단 정당 대표 후보 대선 경선 의원 대통령 기사 정치 선거'.split(' '))

In [ ]:
from collections import Counter
from itertools import chain

all_nouns = tuple(chain(*news_noun_list))

# feature가 될 단어의 조건
# 1. 등장 횟수가 6회 이상
# 2. 단어 길이 2 이상
# 3. 사전에 등록된 불용어(stopword)가 아님
feature_list = [
    word for word, count in Counter(all_nouns).items()
    if count > 5 and len(word) > 1 and word not in stop_list
]

# Set Values

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

# 뉴스별로 단어의 등장여부 딕셔너리 만들기
news_count_list = [Counter(news_noun) for news_noun in news_noun_list]
# feature_list에 등장하는 feature 단어 순으로 등장 횟수 list 만들기
tf = [[count[f] for f in feature_list] for count in news_count_list]

In [ ]:
# np array, 즉, term frequency matrix로 바꾸기
tf = np.array(tf)

In [ ]:
# tf-idf matrix로 바꾸기
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(tf)
type(tfidf)

term frequency matrix는 sparse한, 즉 대부분 성분값이 0인 행렬이다. 따라서 메모리 효율을 위해 sparse matrix 자료구조를 사용한다. 우리가 알고 있는 numpy array 형태로 바꾸자.

In [ ]:
X = tfidf.toarray()
type(X)

### Some functions...

In [ ]:
def get_y(cand):
    return np.array([1 if label==cand else 0 for label in df_filtered['label']])

In [ ]:
def printf_list(num_feature, feature_score, feature_name):
    top_k_indices = np.argsort(-feature_score)[:num_feature]
    for i, k in enumerate(top_k_indices):
        print('{}'.format(feature_name[k]), end=', ')
        if (i+1)%10 == 0:
            print('')

# RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

for cand in cand_list:
    print('--------{}--------'.format(cand))
    clf = RandomForestClassifier(n_estimators=100)
    clf.fit(X, get_y(cand)) # clf.feature_importances_
    printf_list(30, clf.feature_importances_, feature_list)

# LogisticRegression (Lasso)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

for cand in cand_list:
    print('--------{}--------'.format(cand))
    clf = LogisticRegression(penalty='l1', C=1)
    y_true = get_y(cand)
    clf.fit(X, y_true) # clf.coef_
    y_pred = clf.predict(X)
    print(accuracy_score(y_true, y_pred))
    print(np.sum(clf.coef_[0] != 0))
    printf_list(20, clf.coef_[0], feature_list)
    print('-')
    printf_list(20, -clf.coef_[0], feature_list)

# 문재인 vs 안철수

In [ ]:
# df_filtered['label'] != '안철수' 

case = {
    '문재인': 1,
    '홍준표': 0,
}

case_value = np.array(
    [case[label] if label in case else -1 for label in df_filtered['label']]
)

indices = np.argwhere(case_value != -1)
y_true = case_value[indices].reshape(-1)
X_filtered = X[indices].reshape(y_true.shape[0], X.shape[1])

clf = LogisticRegression(penalty='l1', C=1)
clf.fit(X_filtered, y_true) # clf.coef_
y_pred = clf.predict(X_filtered)
print(accuracy_score(y_true, y_pred))
print(np.sum(clf.coef_[0] != 0))
print('--- 문재인 관련 키워드: ')
printf_list(20, clf.coef_[0], feature_list)
print('--- 홍준표 관련 키워드: ')
printf_list(20, -clf.coef_[0], feature_list)

# LDA

In [ ]:
import gensim

In [ ]:
feature_set = set(feature_list)
doc_list = [
    [noun for noun in news if noun in feature_set] for news in news_noun_list
] # news 별로 feature_set에 들어가는 단어만 남기기

In [ ]:
dictionary = gensim.corpora.dictionary.Dictionary(doc_list)
# dictionary 생성

In [ ]:
corpus = [dictionary.doc2bow(nouns) for nouns in doc_list]
# document를 bag-of-words 벡터로 바꿔줌

In [ ]:
lda = gensim.models.ldamodel.LdaModel(
    corpus=corpus, 
    id2word=dictionary, 
    num_topics=100
) # lda modeling

In [ ]:
import pyLDAvis
from pyLDAvis.gensim import prepare

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
pyLDAvis.gensim.prepare(lda, corpus, dictionary)